In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
import numpy as np
import cv2

In [2]:
#値を-1から1に正規化する関数
def normalize_x(image):
    return image / 127.5 - 1


def denormalize_x(image):
    return (image + 1) * 127.5


#値を0から1正規化する関数
def normalize_y(image):
    return image / 255


#値を0から255に戻す関数
def denormalize_y(image):
    return image * 255

In [3]:
def load_Y_gray_with_gaussian_and_numeric(folder_path, thresh = None , normalize = True, g_size = None):
    image_files = []
    #image_files = os.listdir(folder_path)
    
    for file in os.listdir(folder_path):
        base, ext = os.path.splitext(file)
        if ext == '.png':
            image_files.append(int(base))
        else:
            pass
        
    image_files.sort()
    
    img = cv2.imread(folder_path + os.sep + str(image_files[0]).zfill(4) + ".png", cv2.IMREAD_GRAYSCALE)
    
    images = np.zeros(
        (len(image_files), img.shape[0], img.shape[1], 1) ,np.float32
    )
    
    for i , image_file in enumerate(image_files):
        image = cv2.imread(
            folder_path + os.sep + str(image_file).zfill(4) + ".png" ,
            cv2.IMREAD_GRAYSCALE
        )
        #print(image.shape)
        
        # ぼかし処理
        if g_size:
            image = cv2.GaussianBlur(
                image, (g_size, g_size), 0
            )
        
        
        
        if thresh:
            ret , image = cv2.threshold(image , thresh , 255 , cv2.THRESH_BINARY)
        image = image[ : , : , np.newaxis]
        if normalize:
            images[i] = normalize_y(image)
        else:
            images[i] = image
    
    return images , image_files

In [4]:
def culc_F1(pred, gt):
    
    PRE = np.count_nonzero((pred*255 > 0.5) & (gt* 255 > 0.5)) / np.count_nonzero(pred*255 > 0.5)
    RE = np.count_nonzero((pred*255 > 0.5) & (gt * 255 > 0.5)) / np.count_nonzero(gt*255 > 0.5)
    
    try:
        F1 = 2 * (PRE * RE) / (PRE + RE)
        print(f"Precision: {round(PRE,3)}, Recall: {round(RE,3)}, F1: {round(F1,3)} ")
        print(f"Precision: {PRE}, Recall: {RE}, F1: {F1} ")
        return F1
    except ZeroDivisionError:
        return 0
    except:
        return "error"
    
    
def calc_iou(im1, im2):
    intersection = np.logical_and(im1, im2)
    union = np.logical_or(im1, im2)
    iou_score = np.sum(intersection) / np.sum(union)
    if np.sum(union) == 0:
        iou_score = 0
    return round(iou_score,3), iou_score

In [5]:
suga_imgs, _ = load_Y_gray_with_gaussian_and_numeric("I:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/superhuman_dataset/cropped_001/suga_seg/", normalize=False)
nakamura_imgs, _ = load_Y_gray_with_gaussian_and_numeric("Z:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/superhuman_dataset/cropped_001/nakamura_seg//", normalize=False)

lamellar_imgs, _ = load_Y_gray_with_gaussian_and_numeric("I:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/superhuman_dataset/cropped_001/lamellar//", normalize=False)
tubular_imgs, _ = load_Y_gray_with_gaussian_and_numeric("I:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/superhuman_dataset/cropped_001/tubular//", normalize=False)

nakamura_lamellar_imgs = (nakamura_imgs == 1).astype(int)
suga_lamellar_imgs = (suga_imgs == 1).astype(int)

nakamura_tubular_imgs = (nakamura_imgs == 2).astype(int)
suga_tubular_imgs = (suga_imgs == 2).astype(int)

# HITL-TAP → Human＃１

In [6]:
print("HITL-TAPから見たhuman#1 Lamellar")
culc_F1(nakamura_lamellar_imgs, lamellar_imgs)
print("IoU: ", calc_iou(nakamura_lamellar_imgs, lamellar_imgs))

HITL-TAPから見たhuman#1 Lamellar
Precision: 0.734, Recall: 0.849, F1: 0.787 
Precision: 0.7338284394795145, Recall: 0.848698099929627, F1: 0.787094290997632 
IoU:  (0.649, 0.6489327943266325)


In [7]:
print("HITL-TAPから見たhuman#1 Tubular")
culc_F1(nakamura_tubular_imgs, tubular_imgs)
print("IoU: ", calc_iou(nakamura_tubular_imgs, tubular_imgs))

HITL-TAPから見たhuman#1 Tubular
Precision: 0.687, Recall: 0.399, F1: 0.504 
Precision: 0.6866971290628503, Recall: 0.39856885508406725, F1: 0.5043852705569453 
IoU:  (0.337, 0.3372427809297993)


# HITL-TAP → Human＃2

In [8]:
print("HITL-TAPから見たhuman#2 Lamellar")
culc_F1(suga_lamellar_imgs, lamellar_imgs)
print("IoU: ", calc_iou(suga_lamellar_imgs, lamellar_imgs))

HITL-TAPから見たhuman#2 Lamellar
Precision: 0.78, Recall: 0.858, F1: 0.817 
Precision: 0.7795882237230346, Recall: 0.8581533409119616, F1: 0.8169863343153845 
IoU:  (0.691, 0.690597545923183)


In [9]:
print("HITL-TAPから見たhuman#2 Tubular")
culc_F1(suga_tubular_imgs, tubular_imgs)
print("IoU: ", calc_iou(suga_tubular_imgs, tubular_imgs))

HITL-TAPから見たhuman#2 Tubular
Precision: 0.734, Recall: 0.539, F1: 0.622 
Precision: 0.7342514316880283, Recall: 0.5389311449159327, F1: 0.6216091423294471 
IoU:  (0.451, 0.4509672556737292)


# Human#1 → Human#2

In [10]:
print("human#1から見たhuman#2 Lamellar")
culc_F1(suga_lamellar_imgs, nakamura_lamellar_imgs)
print("IoU: ", calc_iou(suga_lamellar_imgs, nakamura_lamellar_imgs))

human#1から見たhuman#2 Lamellar
Precision: 0.8, Recall: 0.761, F1: 0.78 
Precision: 0.7995869123336174, Recall: 0.7610384747402253, F1: 0.7798366080992503 
IoU:  (0.639, 0.6391247379454926)


In [11]:
print("human#1から見たhuman#2 Tubular")
culc_F1(suga_tubular_imgs, nakamura_tubular_imgs)
print("IoU: ", calc_iou(suga_tubular_imgs, nakamura_tubular_imgs))

human#1から見たhuman#2 Tubular
Precision: 0.497, Recall: 0.629, F1: 0.555 
Precision: 0.497204799563676, Recall: 0.6287611000948358, F1: 0.5552975216050557 
IoU:  (0.384, 0.3843680826394013)
